In [1]:
import warnings
import os
import sys

warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))
sys.path.append(parent_dir)

In [2]:
from src.main.models.bidirLSMT import BidirectionalLSTM
from src.main.pipeline.functions import clean_text, stop_words_removal, tokenize
from src.main.utilities.utils import get_dataset
from sklearn.model_selection import train_test_split
from config.config import TENSORBOARD_LOGS
import keras as K

import warnings
warnings.filterwarnings("ignore")

In [3]:
bidirLSTM = BidirectionalLSTM()

bidirLSTM.pipeline = [clean_text, stop_words_removal, tokenize]

inputs, targets = get_dataset(one_hot=True)
inputs = bidirLSTM.run_pipeline(inputs, save=True)

# stratified train test split
x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state=42, stratify=targets)

In [4]:
callbacks = [
    K.callbacks.EarlyStopping(monitor='val_loss', patience=2),
    K.callbacks.TensorBoard(TENSORBOARD_LOGS.format(repr(bidirLSTM)))
]
bidirLSTM.grid_search(x_train, y_train, callbacks=callbacks, validation_split=0.15, n_iter=1, refit=True)

Reloading Tuner from /Users/fabriziodecastelli/Desktop/Magistrale/HLT/HLT-News-Category/results/BidirectionalLSTM/BidirectionalLSTM/tuner0.json
2301/2301 ━━━━━━━━━━━━━━━━━━━━ 113s 48ms/step - accuracy: 0.7555 - loss: 0.6947 - val_accuracy: 0.8691 - val_loss: 0.3963


In [5]:
%load_ext tensorboard
%tensorboard --logdir "{TENSORBOARD_LOGS.format(repr(bidirLSTM))}" --host localhost

In [6]:
bidirLSTM.evaluate(x_test, y_test)

677/677 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8685 - loss: 0.3892


[0.3999994695186615, 0.8650221824645996]

In [10]:
bidirLSTM.save_model()

In [12]:
a = BidirectionalLSTM.load_model()
a.evaluate(x_test, y_test)

677/677 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8685 - loss: 0.3892


[0.3999994695186615, 0.8650221824645996]